### Welcome to MFDM - AI Slamdunk Hackathon . 

#### This hackathon has two parts  

1. Developing basic image transformation operations 
2. Developing convolutional operations 

-----------------------------------------------------------------------------------------------


#### Some Instructions 

1. Kindly follow the naming convention mentioned in the jupyter notebook - else your code will not be testable 
2. Necessary Instructions have been provided
3. Dont forget to run the final cell in the jupyter notebook failing which your efforts will be invalid 
-------------------------------------------------------------------------------------------------

#### Import the necessary packages here

In [132]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import skimage
from numpy.lib.stride_tricks import as_strided



# Define functions for the following tasks using same function and parameter names:
    1. rgb2gray(img) - Converts RGB image to Grayscale
    2. tilt(img, angle) - Rotates the image by the specified angle
    3. horizontal_flip(img) - Flip image along Y - axis
    4. vertical_flip(img) - Flip image along X - axis
    5. crop(img,x1,y1,x2,y2) - Crop out the specified image window
### You may use scikit-image which is preinstalled in this environment

In [133]:
## Function name: rgb2gray
## parameters : img: np.array - Multi Channel
## return-type: img: np.array

### Define your function here (will be evaluated)
def rgb2gray(img):
    return skimage.color.rgb2gray(img) #np.dot(img[...,:3],[0.299,0.587,0.144])

In [134]:
## Function name: tilt
## parameters : img: np.array
##              angle: int - 0 to 360
## return-type: img: np.array

### Define your function here (will be evaluated)
def tilt(img, angle):
    return skimage.transform.rotate(img,angle) #ndimage.rotate(img,angle)

In [135]:
## Function name: horizontal_flip
## parameters : img: np.array
## return-type: img: np.array

### Define your function here (will be evaluated)
def  horizontal_flip(img):
    return np.fliplr(img)

In [136]:
## Function name: vertical_flip
## parameters : img: np.array
## return-type: img: np.array

### Define your function here (will be evaluated)
def  vertical_flip(img):
    return np.flipud(img)

In [137]:
## Function name: crop
## parameters : img: np.array - Single Channel or Multi Channel
##              x1: int - Window X_min
##              y1: int - Window Y_min
##              x2: int - Window X_max
##              y2: int - Window Y_max
## return-type: img: np.array

### Define your function here (will be evaluated)
def crop(img,x1,y1,x2,y2):
    if(len(img.shape)==3):
        return img[x1:x2,y1:y2,:]
    else:
        return img[x1:x2,y1:y2]

### Implement the convolution operations using numpy as per the instructions given below

- You are only allowed to use Numpy package to perform convolution operations, please avoid using other packages, doing so you will be disqualified.

In [138]:
## Function name: convolution
## parameters : img: np.array - Single Channel or Multi Channel
##              layer_filters: np.array - Shape: (num_filters, filter_height, filter_width, num_channels) if Multichannel else
##                                        Shape: (num_filters, filter_height, filter_width) if Singlechannel
##              padding: str - 'same' or 'valid', default = 'valid'
## return-type: img: np.array

### Define your function here (will be evaluated)

def convolution_intern(img, layer_filters, padding):
    filter_size = layer_filters.shape[1]
    result = np.zeros((img.shape))
    for r in np.uint16(np.arange(filter_size/2.0, 
                          img.shape[0]-filter_size/2.0+1)):
        for c in np.uint16(np.arange(filter_size/2.0, 
                                           img.shape[1]-filter_size/2.0+1)):
            curr_region = img[r-np.uint16(np.floor(filter_size/2.0)):r+np.uint16(np.ceil(filter_size/2.0)), 
                              c-np.uint16(np.floor(filter_size/2.0)):c+np.uint16(np.ceil(filter_size/2.0))]
            
            curr_result = curr_region * layer_filters
            convolution_internsum = np.sum(curr_result)
            result[r, c] = convolution_internsum
            
    #Clipping the outliers of the result matrix.
    final_result = result[np.uint16(filter_size/2.0):result.shape[0]-np.uint16(filter_size/2.0), 
                          np.uint16(filter_size/2.0):result.shape[1]-np.uint16(filter_size/2.0)]
    return final_result
    #return result
def convolution(img, layer_filters, padding='valid'):
    
    feature_maps = np.zeros((img.shape[0]-layer_filters.shape[1]+1, 
                                img.shape[1]-layer_filters.shape[1]+1, 
                                layer_filters.shape[0]))

    # Convolving the image by the filter(s).
    for filter_num in range(layer_filters.shape[0]):
        curr_filter = layer_filters[filter_num, :]
        
        if len(curr_filter.shape) > 2:
            convolution_internmap = convolution_intern(img[:, :, 0], curr_filter[:, :, 0], padding) 
            for ch_num in range(1, curr_filter.shape[-1]):
                convolution_internmap = convolution_internmap + convolution_intern(img[:, :, ch_num], 
                                  curr_filter[:, :, ch_num], padding)
        else: convolution_internmap = convolution_intern(img, curr_filter, padding)
        feature_maps[:, :, filter_num] = convolution_internmap
    return feature_maps



### Sample IO

### GrayScale Image    (use this as a sample image to test your functions)
<img src = 'resz_gray.png'>

#### image location resz_gray.png in the current directory

In [139]:
### Sample filters to test your output, use plt.imshow() to output the resulting image

num_filters = 4
filter_h = 3
filter_w = 3

layer1_filter = np.zeros((num_filters, filter_h, filter_w))

#Vertical
layer1_filter[0,:,:] = np.array([
    [
        [-1,0,1],
        [-1,0,1], 
        [-1,0,1]
    ]
])
#Horizontal
layer1_filter[1,:,:] = np.array([
    [
        [1,1,1],
        [0,0,0],
        [-1,-1,-1]
    ]
])
#Blur
layer1_filter[2,:,:] = np.array([
    [
        [1/1e9,1/1e9,1/1e9],
        [1/1e9,1/1e9,1/1e9],
        [1/1e9,1/1e9,1/1e9]
    ]
])
#Sharpen
layer1_filter[3,:,:] = np.array([
    [
        [0,-1,0],
        [-1,5,-1],
        [0,-1,0]
    ]
])
"""
Use the above filters to test your convolution function on variaous filter defined above.
use plt.imshow() on the resulting n-d array to test the output against the expected output 
shown in the below cell.
"""

'\nUse the above filters to test your convolution function on variaous filter defined above.\nuse plt.imshow() on the resulting n-d array to test the output against the expected output \nshown in the below cell.\n'

## Convolution  (expected output after convolution)
    Input:Single channel image, padding='same'
<img src = 'post_conv.png'>

In [140]:
## Function name: relu
## parameters : img: np.array - Single Channel or Multi Channel
## return-type: img: np.array

### Define your function here (will be evaluated)
def relu(img):
    relu_out = np.zeros(img.shape)
    for map_num in range(img.shape[-1]):
        for r in np.arange(0,img.shape[0]):
            for c in np.arange(0, img.shape[1]):
                relu_out[r, c, map_num] = np.max([img[r, c, map_num], 0])
    return relu_out



### Output image after applying Relu
    Input: Images after convolution
<img src = 'post_relu.png'>

In [141]:
## Function name: pooling
## parameters : img: np.array - Single Channel or Multi Channel
##              size: int - default=2
##              stride: int - default = 2
##              mode: str = "max" or "avg" - default = 'max'
## return-type: img: np.array


### Define your function here (will be evaluated)
def pooling(img, size=2, stride=2, mode='max'):
    pool_out = np.zeros((np.uint16((img.shape[0]-size+1)/stride+1),
                            np.uint16((img.shape[1]-size+1)/stride+1),
                            img.shape[-1]))
    for map_num in range(img.shape[-1]):
        r2 = 0
        for r in np.arange(0,img.shape[0]-size+1, stride):
            c2 = 0
            for c in np.arange(0, img.shape[1]-size+1, stride):
                if(mode=='max'):
                    pool_out[r2, c2, map_num] = np.max([img[r:r+size,  c:c+size, map_num]])
                else:
                    pool_out[r2, c2, map_num] = np.mean([img[r:r+size,  c:c+size, map_num]])
                c2 = c2 + 1
            r2 = r2 +1
    return pool_out



### Output image after applying Pooling
    Input: Images after relu, size = 3, stride = 3, mode = 'max'
<img src = 'post_pool.png'>

### Once you are done with your solution run the below cell before running the test cases

In [142]:
import os
if __name__=='__main__':
    !jupyter nbconvert --to script CNN_Scratch_Q.ipynb
    

    cwd = os.getcwd()

    pwd = os.path.dirname(cwd)

    os.rename(cwd+"/CNN_Scratch_Q.py", pwd+"/contestant.py")

[NbConvertApp] Converting notebook CNN_Scratch_Q.ipynb to script
[NbConvertApp] Writing 8953 bytes to CNN_Scratch_Q.py
